In [15]:
#Here you must specify your own path
root = ''

In [16]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFECV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [17]:
# Dic to get the respective target values of the numbers.
target_dic = {
    0: '(0, 20]constant',
    1: '(20, 40]constant',
    2: '(40, 60]constant',
    3: '(60, 80]constant',
    4: '(80, 100]constant',
    5: '(0, 20]intermittent',
    6: '(20, 40]intermittent',
    7: '(40, 60]intermittent',
    8: '(60, 80]intermittent',
    9: '(80, 100]intermittent',
}

In [18]:
#read in csv file into dataframe
df_train = pd.read_csv(root + 'data/02_processed/train.csv', sep=';')
df_train.pop('Unnamed: 0')

df2=df_train.copy()
df2.drop(['target','4','66','67'], axis = 1,inplace=True)


#read validation and test data from csv
df_validation = pd.read_csv(root + 'data/02_processed/validation.csv', sep=';')
df_validation.set_index('user_id',inplace=True)
df_test = pd.read_csv(root + 'data/02_processed/test.csv', sep=';')
df_test.set_index('user_id',inplace=True)

#encode target column of test data
target_test=pd.get_dummies(df_test['target'], prefix='target')

#at the end, the 3 data frames will be written into several CSV.
completed_train=df_train.copy()
completed_test=df_test.copy()
completed_validation=df_validation.copy()

# target of train data without encoding
target_train= df_train['target']

#delete target from training data
df_train.pop('target')

# target of test data without encoding
target_test= df_test['target']

#delete target from test data
df_test.pop('target')

#Will be used for training the RFECV.
train=df_train.copy()



train2=df_train.copy()

In [19]:
#extract numeric columns and encode categorical columns with  one hot encoding.
toOneHot = OneHotEncoder(handle_unknown='ignore')
categorical = [df2.columns]
categorical
column_4 =train2['4']
column_66 =train2['66']
column_67 =train2['67']

for index in categorical:
    train2= pd.get_dummies(train[index],prefix=index)
    
    
train2['4']=column_4
train2['66']=column_66
train2['67']=column_67

train2


,5_Female,5_Male,6_Both Sides,6_Left,6_Right,8_No,8_children,8_parents,8_parents|children,8_parents|siblings,...,82_YES,83_NO,83_YES,84_NO,84_YES,85_NO,85_YES,4,66,67
0,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0.230769,0.761029,0.544118
1,1,0,0,0,1,1,0,0,0,0,...,0,1,0,1,0,1,0,0.296703,0.170000,0.060000
2,1,0,1,0,0,1,0,0,0,0,...,1,1,0,1,0,1,0,0.428571,0.920000,0.890000
3,1,0,0,0,1,0,0,1,0,0,...,1,1,0,1,0,1,0,0.329670,0.533088,0.591912
4,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,1,0,0.670330,0.560000,0.540000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,1,0,0,0,1,1,0,0,0,0,...,0,0,1,1,0,1,0,0.692308,0.330000,0.130000
3166,1,0,0,0,1,1,0,0,0,0,...,0,0,1,1,0,1,0,0.692308,0.330000,0.130000
3167,1,0,0,0,1,1,0,0,0,0,...,0,0,1,1,0,1,0,0.692308,0.330000,0.130000
3168,1,0,0,1,0,1,0,0,0,0,...,0,0,1,1,0,1,0,0.527473,0.900000,0.940000


In [20]:
#categorical coumn names
categorical_columns = ['5', '6', '8', '10', '11', '13', '35', '37', '40',
       '65', '68', '69', '70', '71', '72', '73', '74', '76', '77', '78', '79',
       '80', '81', '82', '83', '84', '85']
        
#numeric column names
numerical_columns = ['4','66','67']

In [21]:
#OneHotEncoder
categorical_encoder = OneHotEncoder(handle_unknown='ignore')

#DecisionTreeClassifier
Dtc = DecisionTreeClassifier(random_state=0)



#Scorer to measure the performance of the estimators.
scorer = make_scorer(f1_score,average='macro')

#ColumnTransformer encodes categorical data with the categorical_encoder.
#and numeric columns are not edited.
preprocessing = ColumnTransformer(
    [('cat', categorical_encoder, categorical_columns),
     ('num', 'passthrough', numerical_columns)])



#Created Pipeline to combine preprocessing of the data and the RFECV.
#RFECV eliminates features recursively so that the useful features for the models remain.
rfcv = Pipeline([
    ('preprocess', preprocessing),
    ('classifier',RFECV(estimator=Dtc,scoring=scorer,n_jobs=-1,verbose=3))
])

rfcv.fit(train, target_train)

Fitting estimator with 89 features.
Fitting estimator with 88 features.
Fitting estimator with 87 features.
Fitting estimator with 86 features.
Fitting estimator with 85 features.
Fitting estimator with 84 features.
Fitting estimator with 83 features.
Fitting estimator with 82 features.
Fitting estimator with 81 features.
Fitting estimator with 80 features.
Fitting estimator with 79 features.
Fitting estimator with 78 features.
Fitting estimator with 77 features.
Fitting estimator with 76 features.
Fitting estimator with 75 features.
Fitting estimator with 74 features.
Fitting estimator with 73 features.
Fitting estimator with 72 features.
Fitting estimator with 71 features.
Fitting estimator with 70 features.
Fitting estimator with 69 features.
Fitting estimator with 68 features.
Fitting estimator with 67 features.
Fitting estimator with 66 features.
Fitting estimator with 65 features.
Fitting estimator with 64 features.
Fitting estimator with 63 features.
Fitting estimator with 62 fe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['5', '6', '8', '10', '11',
                                                   '13', '35', '37', '40', '65',
                                                   '68', '69', '70', '71', '72',
                                                   '73', '74', '76', '77', '78',
                                                   '79', '80', '81', '82', '83',
                                                   '84', '85']),
                                                 ('num', 'passthrough',
                                                  ['4', '66', '67'])])),
                ('classifier',
                 RFECV(estimator=DecisionTreeClassifier(random_state=0),
                       n_jobs=-1, scoring=make_scorer(f1_score, average=macro),
                       ver

In [22]:
#Get a boolean mask of the features which were selected by the RFECV.
mask = rfcv.named_steps['classifier'].support_
rfcv.named_steps['preprocess'].named_transformers_['cat'].categories_

[array(['Female', 'Male'], dtype=object),
 array(['Both Sides', 'Left', 'Right'], dtype=object),
 array(['No', 'children', 'parents', 'parents|children',
        'parents|siblings', 'siblings'], dtype=object),
 array(['Abrupt', 'Gradual'], dtype=object),
 array(['change in hearing', 'head trauma', 'loud blast of sound', 'other',
        'stress', 'whiplash'], dtype=object),
 array(['NO', 'YES with heart beat', 'YES, different from heart beat'],
       dtype=object),
 array(['both ears, equally', 'both ears, worse in left',
        'both ears, worse in right', 'elsewhere', 'inside the head',
        'left ear', 'right ear'], dtype=object),
 array(['No', 'Yes'], dtype=object),
 array(['crickets', 'noise', 'other', 'tone'], dtype=object),
 array(['high frequency', 'low frequency', 'medium frequency',
        'very high frequency'], dtype=object),
 array(['2 - 4', '5 and more', 'none', 'one'], dtype=object),
 array(['I don’t know', 'NO', 'YES'], dtype=object),
 array(['I don’t know', 'NO',

In [23]:
#empty dataframe
data = pd.DataFrame()

#Get the one hot encoded column names.
column_names= train2.columns

#List which will be filled with  the column names of the features 
#which were selected by the RFECV and a target label
Vals = ['target']

In [24]:
#iterate trough mask to get the selected features and write column names into data.
for i in range(0,89):
    if mask[i]==True:
        data[column_names[i]]=train2.iloc[:,i]
        
data.columns       

Index(['5_Male', '8_No', '10_Abrupt', '11_stress', '35_both ears, equally',
       '35_both ears, worse in left', '37_Yes', '40_tone', '68_2 - 4',
       '68_none', '68_one', '70_NO', '71_YES', '72_worsens my tinnitus',
       '74_has no effect', '76_YES', '78_Never', '78_Sometimes', '79_NO',
       '80_YES', '83_YES', '4', '66', '67'],
      dtype='object')

In [25]:
#Write column names
for i in range(0,len(data.columns)):
    Vals.insert(i,data.columns[i].split('_')[0])

#remove duplicates    
Vals = list(dict.fromkeys(Vals))

#filter the DataFrames so that only the column names from Vals are included.
completed_train= completed_train.filter(items=Vals)                                  
completed_validation= completed_validation.filter(items=Vals)
completed_test= completed_test.filter(items=Vals)       

In [26]:
#Train Test and Validation Data is written into different CSV.
completed_train.to_csv(root+"data/02_processed/completed_train.csv",sep=';')
completed_test.to_csv(root+"data/02_processed/completed_test.csv",sep=';')
completed_validation.to_csv(root+"data/02_processed/completed_validation.csv",sep=';')

In [27]:
#The Columns 6 13 65 69 73 77 81 82 84 85 are discarded because of RFECV.
completed_train.columns

Index(['5', '8', '10', '11', '35', '37', '40', '68', '70', '71', '72', '74',
       '76', '78', '79', '80', '83', '4', '66', '67', 'target'],
      dtype='object')

In [28]:
print('reached')

reached
